## Problem 2: Investigation of Life Expectancy 

### 1. Report the summary of the linear model. What are the predicting variables actually affecting the life expectancy? Justify your answer based on the outputs of linear regression model.

In [ ]:
import pandas as pd
import statsmodels.api as sm

# definition
file_path = "/root/MSDM5054/Life Expectancy Data.csv"
life_exp_df = pd.read_csv(file_path)

# Cleanup
life_exp_df.columns = life_exp_df.columns.str.strip()

# Preprocessing
life_exp_df.drop('Country', axis=1, inplace=True)

life_exp_df.dropna(subset=['Life expectancy'], inplace=True)

life_exp_df['Status'] = life_exp_df['Status'].map({'Developing': 1, 'Developed': 0})

# definition
target_variable = life_exp_df['Life expectancy']
predictor_variables = life_exp_df.drop('Life expectancy', axis=1)

# Feature filling
predictors_imputed = predictor_variables.apply(lambda col: col.fillna(col.mean()))

predictors_with_const = sm.add_constant(predictors_imputed)

# （OLS）
regression_model = sm.OLS(target_variable, predictors_with_const).fit()

print(regression_model.summary())

                            OLS Regression Results                            
Dep. Variable:        Life expectancy   R-squared:                       0.820
Model:                            OLS   Adj. R-squared:                  0.819
Method:                 Least Squares   F-statistic:                     664.0
Date:                Wed, 24 Sep 2025   Prob (F-statistic):               0.00
Time:                        14:34:31   Log-Likelihood:                -8239.5
No. Observations:                2928   AIC:                         1.652e+04
Df Residuals:                    2907   BIC:                         1.665e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const     

### 2. Construct the 95% confidence intervals for the coefficient of “Adult Mortality” and “HIV/AIDS”. Are you confident that these predictors have negative impact on the life expectancy? Explain why.

In [ ]:

# definition
variables_to_analyze = ['Adult Mortality', 'HIV/AIDS']

# Confidence interval
confidence_intervals_df = regression_model.conf_int(alpha=0.05)

# result
analysis_results = {}

for var_name in variables_to_analyze:
    # Confidence interval
    interval = confidence_intervals_df.loc[var_name].tolist()
    
    # Determining negative impact
    is_negative_impact = interval[1] < 0
    analysis_results[var_name] = {
        'ci': interval,
        'has_negative_impact': is_negative_impact
    }

print("\nAnalysis of 95% Confidence Intervals:")
for var_name, result in analysis_results.items():
    print(f"\nVariable: {var_name}")
    print(f"  - Confidence Interval: {result['ci']}")
    if result['has_negative_impact']:
        print(f"  - Conclusion: Confident that it has a negative impact on life expectancy.")
    else:
        print(f"  - Conclusion: Cannot be confident of a negative impact on life expectancy.")


Analysis of 95% Confidence Intervals:

Variable: Adult Mortality
  - Confidence Interval: [-0.021350525396991272, -0.018228358141137355]
  - Conclusion: Confident that it has a negative impact on life expectancy.

Variable: HIV/AIDS
  - Confidence Interval: [-0.5053295472183662, -0.4360966923108973]
  - Conclusion: Confident that it has a negative impact on life expectancy.


### 3. Construct the 97% confidence intervals for the coefficient of “Schooling” and “Alcohol”. Explain how these predictors impact the life expectancy.

In [ ]:
alpha = 0.03
variables_of_interest = ['Schooling', 'Alcohol']

# Calculating confidence intervals
ci_dataframe = regression_model.conf_int(alpha=alpha)

print(f"\n--- Analysis at {(1-alpha)*100:.0f}% Confidence Level ---")

for var in variables_of_interest:
    # confidence intervals
    interval = ci_dataframe.loc[var]
    
    # print intervals
    print(f"\n{var} {(1-alpha)*100:.0f}% confidence intervals:", interval.values)
    
    lower_bound, upper_bound = interval
    
    if lower_bound > 0 and upper_bound > 0:
        print(f"{var} has positive impact on the life expectancy.")
    elif lower_bound < 0 and upper_bound < 0:
        print(f"{var} has negative impact on the life expectancy.")
    else:
        print(f"{var} has no impact on the life expectancy.")


--- Analysis at 97% Confidence Level ---

Schooling 97% confidence intervals: [0.58155514 0.76666183]
Schooling has positive impact on the life expectancy.

Alcohol 97% confidence intervals: [0.00418148 0.11756867]
Alcohol has positive impact on the life expectancy.


### 4. Based on the p-values, which are the top-seven most influential predictors? Use these predictors to fit a smaller model and report the summary.

In [ ]:
# The 7 most important predictors
top_features_list = (regression_model.pvalues
                     .drop('const')
                     .sort_values()
                     .head(7)
                     .index.tolist())

print("\ntop7_vars:", top_features_list)

# Building a model
X_reduced = predictors_with_const.loc[:, top_features_list + ['const']]

# summary
print(sm.OLS(target_variable, X_reduced).fit().summary())


top7_vars: ['HIV/AIDS', 'Adult Mortality', 'Schooling', 'under-five deaths', 'infant deaths', 'BMI', 'Income composition of resources']
                            OLS Regression Results                            
Dep. Variable:        Life expectancy   R-squared:                       0.791
Model:                            OLS   Adj. R-squared:                  0.791
Method:                 Least Squares   F-statistic:                     1583.
Date:                Wed, 24 Sep 2025   Prob (F-statistic):               0.00
Time:                        15:02:24   Log-Likelihood:                -8458.3
No. Observations:                2928   AIC:                         1.693e+04
Df Residuals:                    2920   BIC:                         1.698e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|

In [ ]:
reduced_model = sm.OLS(target_variable, X_reduced).fit()

### 5. Use the smaller model to predict the life expectancy if a new observation is given with Year=2008, Status=Developed, Adult Mortality=125, infant deaths=94, Alcohol=4.1, percentage expenditure=100, Hepatitis B=20, Measles=13, BMI=55, under-five deaths=2, Polio=12, Total expenditure=5.9, Diphtheria=12, HIV/AIDS=0.5, GDP=5892,Population=1.34e6, Income composition of resources=0.9, Schooling=18. Report the 99% confidence interval for your prediction.

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm


# Training mean
training_means = predictor_variables.mean()

# definition
new_observation = {
    'Year': 2008,
    'Status': 1,  # (1=Developing, 0=Developed)
    'Adult Mortality': 125,
    'infant deaths': 94,
    'Alcohol': 4.1,
    'percentage expenditure': 100,
    'Hepatitis B': 20,
    'Measles': 13,
    'BMI': 55,
    'under-five deaths': 2,
    'Polio': 12,
    'Total expenditure': 5.9,
    'Diphtheria': 12,
    'HIV/AIDS': 0.5,
    'GDP': 5892,
    'Population': 1.34e6,
    'thinness 1-19 years': np.nan, # Missing Values
    'thinness 5-9 years': np.nan,  # Missing Values
    'Income composition of resources': 0.9,
    'Schooling': 18
}

# preparation
new_obs_df = pd.DataFrame([new_observation])

# filling
new_obs_df.fillna(training_means, inplace=True)

# c. Add a constant term
new_obs_df_with_const = sm.add_constant(new_obs_df, has_constant='add')

X_new_final = new_obs_df_with_const[top_features_list + ['const']]

# prediction 
prediction = reduced_model.get_prediction(X_new_final)

pred_summary = prediction.summary_frame(alpha=0.01)

print("\nPrediction Summary for New Observation (with 99% Intervals):")
print(pred_summary)


Prediction Summary for New Observation (with 99% Intervals):
        mean   mean_se  mean_ci_lower  mean_ci_upper  obs_ci_lower  \
0  89.576442  0.794559      87.528455      91.624429     78.167474   

   obs_ci_upper  
0    100.985409  


### 6. Use AIC to compare the full model and the smaller model.

In [15]:
regression_model_aic = regression_model.aic
reduced_model_aic = reduced_model.aic

print("\nregression_model_aic:", regression_model_aic)
print("reduced_model_aic:", reduced_model_aic)

if reduced_model_aic < regression_model_aic:
    print("\nreduced_model_aic is lower.")
else:
    print("\nregression_model_aic is lower.")


regression_model_aic: 16521.085082746766
reduced_model_aic: 16932.62646207667

regression_model_aic is lower.


## Problem 3: Implementing KNN regression

In [ ]:
import numpy as np
import pandas as pd
import time
from math import sqrt

class KNN:

    def __init__(self, k=5, standardize=False):
        self.k = k
        self.standardize = standardize
        self.X_train = None
        self.y_train = None
        self.train_mean = None
        self.train_std = None

    def fit(self, X, y):
        self.y_train = y
        if self.standardize:
            self.train_mean = np.mean(X, axis=0)
            self.train_std = np.std(X, axis=0)
            self.train_std[self.train_std == 0] = 1 
            self.X_train = (X - self.train_mean) / self.train_std
        else:
            self.X_train = X

    def predict(self, X_test):
        X_test_processed = X_test
        if self.standardize:
            X_test_processed = (X_test - self.train_mean) / self.train_std
        
        predictions = []
        for row in X_test_processed:
            distances = np.linalg.norm(self.X_train - row, axis=1)
            neighbor_indices = np.argsort(distances)[:self.k]
            prediction = np.mean(self.y_train[neighbor_indices])
            predictions.append(prediction)
        return np.array(predictions)

def run_knn_evaluations(X_train, y_train, X_test, y_test, k_options, standardize_flag):
    print(f"=== {'With' if standardize_flag else 'Without'} Standardization ===")
    
    best_k = -1
    best_mse = float('inf')
    
    for k in k_options:
        model = KNN(k=k, standardize=standardize_flag)
        model.fit(X_train, y_train)
        
        start_time = time.time()
        y_pred = model.predict(X_test)
        run_time = time.time() - start_time
        
        mse = np.mean((y_test - y_pred)**2)
        
        if mse < best_mse:
            best_mse = mse
            best_k = k
            
        print(f"K={k}: MSE={mse:.4f}, Time={run_time:.4f} seconds")
        
    print(f"Best K ({'With' if standardize_flag else 'No'} Standardization): {best_k} with MSE={best_mse:.4f}")
    return best_k, best_mse

def main():
    train_df = pd.read_csv('boston_housing_train.csv')
    test_df = pd.read_csv('boston_housing_test.csv')
    X_train = train_df.drop('medv', axis=1).values
    y_train = train_df['medv'].values
    X_test = test_df.drop('medv', axis=1).values
    y_test = test_df['medv'].values
    
    k_values_to_test = list(range(1, 21))
    
    _, best_mse_raw = run_knn_evaluations(X_train, y_train, X_test, y_test, k_values_to_test, standardize_flag=False)
    print()
    _, best_mse_std = run_knn_evaluations(X_train, y_train, X_test, y_test, k_values_to_test, standardize_flag=True)
    
    print("Standardization improved the performance." if best_mse_std < best_mse_raw else "Standardization did not improve the performance.")

if __name__ == '__main__':
    main()

=== Without Standardization ===
K=1: MSE=44.5171, Time=0.0138 seconds
K=2: MSE=46.0559, Time=0.0695 seconds
K=3: MSE=41.5232, Time=0.0132 seconds
K=4: MSE=40.8879, Time=0.0132 seconds
K=5: MSE=42.2411, Time=0.0127 seconds
K=6: MSE=43.8894, Time=0.0688 seconds
K=7: MSE=43.9851, Time=0.0130 seconds
K=8: MSE=42.8303, Time=0.0071 seconds
K=9: MSE=44.0434, Time=0.0714 seconds
K=10: MSE=45.6143, Time=0.0077 seconds
K=11: MSE=45.7835, Time=0.0075 seconds
K=12: MSE=45.8784, Time=0.0076 seconds
K=13: MSE=45.7650, Time=0.0076 seconds
K=14: MSE=46.5139, Time=0.0076 seconds
K=15: MSE=46.5349, Time=0.0642 seconds
K=16: MSE=48.1896, Time=0.0076 seconds
K=17: MSE=49.1327, Time=0.0077 seconds
K=18: MSE=48.9265, Time=0.0093 seconds
K=19: MSE=50.1288, Time=0.0066 seconds
K=20: MSE=51.0900, Time=0.0059 seconds
Best K (No Standardization): 4 with MSE=40.8879

=== With Standardization ===
K=1: MSE=25.4693, Time=0.0625 seconds
K=2: MSE=16.7776, Time=0.0060 seconds
K=3: MSE=19.7349, Time=0.0059 seconds
K=4: 